<a href="https://colab.research.google.com/github/programmerfaz/Machine-learning-Projects/blob/main/Fake_News_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Fake News Detection Using Logistic Regression**

Fake news is a growing problem in the digital world, spreading misinformation and influencing public opinion. Machine learning can help address this issue by automatically classifying news articles as real or fake. This project focuses on building a Logistic Regression model to predict fake news using a labeled dataset.

The dataset contains news articles with labels (real or fake). The process involves preprocessing the text, extracting features (e.g., word frequencies), and training the model to classify new articles. Logistic Regression is a simple yet effective algorithm for binary classification tasks like this.

About the Dataset



1.   id: unique id for a news article
2.   title: the title of a news article
3.   author: author of the news article
4.   text: the text of the article could be incomplete
5.   label: a label that marks whether the news article is real or fake.



1.   0: Real news
2.   1: Fake news





Importing the dependencies

In [2]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords #used for removing words like where, what etc or words that dont add any relevance to the data to be trained
from nltk.stem.porter import PorterStemmer #takes a word and removes all the suffix and prefix of it
from sklearn.feature_extraction.text import TfidfVectorizer #used for converting the features into numbers
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
#listing down some of the stopwords
print(stopwords.words("english"))
#we will remove all these unecessary words that are to be included in our training dataset

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Pre Processing

In [7]:
news_dataset = pd.read_csv("/content/drive/MyDrive/DATASET/fake-news/train.csv")

In [8]:
news_dataset.shape

(20800, 5)

In [11]:
#printing the first 5 values from the dataset
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [13]:
#counting the number of missing values in the dataset
news_dataset.isnull().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [14]:
#dropping the missing values with null string because the data is very large
news_dataset = news_dataset.fillna("")

In [15]:
#Now combining the title and author column which are 2 important feature used for training the logistic regression model
news_dataset["content"] = news_dataset["author"]+" "+news_dataset["title"]

In [17]:
news_dataset.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [18]:
# seaparating the data and label
X = news_dataset.drop(columns="label",axis=1)
Y =news_dataset["label"]

In [19]:
print(X)

          id  ...                                            content
0          0  ...  Darrell Lucus House Dem Aide: We Didn’t Even S...
1          1  ...  Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2          2  ...  Consortiumnews.com Why the Truth Might Get You...
3          3  ...  Jessica Purkiss 15 Civilians Killed In Single ...
4          4  ...  Howard Portnoy Iranian woman jailed for fictio...
...      ...  ...                                                ...
20795  20795  ...  Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796  20796  ...  Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797  20797  ...  Michael J. de la Merced and Rachel Abrams Macy...
20798  20798  ...  Alex Ansary NATO, Russia To Hold Parallel Exer...
20799  20799  ...            David Swanson What Keeps the F-35 Alive

[20800 rows x 5 columns]


In [20]:
print(Y)

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64


**Stemming**

It is the process of reducing a word to its root word.

Example :actor,actress,acting --> act

In stemming we reduce a string word to a numerical values

In [21]:
port_stem = PorterStemmer()

In [29]:
def stemming(content):
  stemmed_content = re.sub("[^a-zA-Z]"," ",content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words("english")]
  stemmed_content = " ".join(stemmed_content) #to convert the list to a string
  return stemmed_content

In [30]:
news_dataset["content"] = news_dataset["content"].apply(stemming)

In [32]:
print(news_dataset["content"])
#all the significant words have been removed from the dataset

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [34]:
#Separating the data and the label
X = news_dataset["content"].values
Y = news_dataset["label"].values
#using feature selection to avoid the problem of overfitting and hence increasing the variance which will eventually lead to less accurate model

In [35]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [36]:
print(Y)

[1 0 1 ... 0 1 1]


In [37]:
#converting the text data into numerical values, using vectorizer vector to do the same
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [38]:
print(X)

  (0, 267)	0.2701012497770876
  (0, 2483)	0.36765196867972083
  (0, 2959)	0.24684501285337127
  (0, 3600)	0.3598939188262558
  (0, 3792)	0.27053324808454915
  (0, 4973)	0.23331696690935097
  (0, 7005)	0.2187416908935914
  (0, 7692)	0.24785219520671598
  (0, 8630)	0.2921251408704368
  (0, 8909)	0.36359638063260746
  (0, 13473)	0.2565896679337956
  (0, 15686)	0.2848506356272864
  (1, 1497)	0.2939891562094648
  (1, 1894)	0.15521974226349364
  (1, 2223)	0.3827320386859759
  (1, 2813)	0.19094574062359204
  (1, 3568)	0.26373768806048464
  (1, 5503)	0.7143299355715573
  (1, 6816)	0.1904660198296849
  (1, 16799)	0.30071745655510157
  (2, 2943)	0.3179886800654691
  (2, 3103)	0.46097489583229645
  (2, 5389)	0.3866530551182615
  (2, 5968)	0.3474613386728292
  (2, 9620)	0.49351492943649944
  :	:
  (20797, 3643)	0.2115550061362374
  (20797, 7042)	0.21799048897828685
  (20797, 8364)	0.22322585870464115
  (20797, 8988)	0.36160868928090795
  (20797, 9518)	0.29542040034203126
  (20797, 9588)	0.17455348

Splitting the datasrt into training and testing data

In [39]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

In [41]:
print(X.shape,X_train.shape,X_test.shape)

(20800, 17128) (16640, 17128) (4160, 17128)


In [42]:
print(Y.shape,Y_train.shape,Y_test.shape)

(20800,) (16640,) (4160,)


Training the model

In [43]:
logistic_regression = LogisticRegression()

In [44]:
logistic_regression.fit(X_train,Y_train)

LogisticRegression()

Evaluating the trained model

In [45]:
#accuracy score on the training data
X_train_prediction  = logistic_regression.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction,Y_train)

In [47]:
print(training_data_accuracy*100)

98.6358173076923


In [48]:
#accuracy score on the testing data
X_test_prediction = logistic_regression.predict(X_test)
testing_data_accuracy = accuracy_score(X_test_prediction,Y_test)

In [50]:
print(testing_data_accuracy*100)

97.90865384615385


Making a predictive system

In [62]:
X_new = X_test[40]

prediction = logistic_regression.predict(X_new)
print(Y_test[0])
print(prediction[0])

if (prediction[0]==0):
  print("Real news")
else:
  print("Fake news")

1
1
Fake news


In [61]:
X_new = X_test[40]

prediction = logistic_regression.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print("Real news")
else:
  print("Fake news")

[1]
Fake news
